In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

import pandas as pd

import os

In [2]:
llm = ChatOllama(model='llama3.2', temperature=0, num_gpu=1)

In [3]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

C:\Users\chand\.conda\envs\fusgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load from local storage
vectorstore = FAISS.load_local("updated_faiss_index_", embeddings,allow_dangerous_deserialization=True)

In [5]:
template = """ Answer the question based only on the following context. You have the freedom to interpret the context however you want, At the end, you have to mention correct option.
        Given below is the context and question of the user.
        Question = {question}
        Context = {context}
        Answer:
         """
prompt = ChatPromptTemplate.from_template(template)

In [6]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["question"])
    # print(retrieved_docs)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [7]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [8]:
response = graph.invoke({"question": "The triple therapy consisting of PPI, clarithromycin, and amoxicillin was administered as an eradication treatment for H. pylori. Which of the following tests is the most appropriate for determining the success of eradication?\n1. H. pylori culture test\n2. H. pylori serology test\n3. Urea breath test\n4. Endoscopic biopsy\n5. Rapid urease test"})
print(response["answer"])

Based on the context provided, the most appropriate test for determining the success of eradication of H. pylori is the Urea breath test (Option 3).

The text states that "Tests have been developed (PyloriTek, CLOtest, Hpfast, Pronto Dry) that have a sensitivity and specificity of >90–95%." The Urea breath test is mentioned as one of these tests, which suggests it has high sensitivity and specificity for detecting H. pylori.

Additionally, the text notes that serologic testing (Option 2) can be affected by recent use of PPIs, antibiotics, or bismuth compounds, making it less reliable in this context. Histology (Option 4) requires pathology processing and staining, which may not be feasible for routine monitoring of eradication success. Culture (Option 1) is time-consuming, expensive, and dependent on experience, making it less practical for regular use.

Therefore, the Urea breath test is the most appropriate option for determining the success of H. pylori eradication treatment.

Corre

In [9]:
questions = [
    "The triple therapy consisting of PPI, clarithromycin, and amoxicillin was administered as an eradication treatment for H. pylori. Which of the following tests is the most appropriate for determining the success of eradication?\n1. H. pylori culture test\n2. H. pylori serology test\n3. Urea breath test\n4. Endoscopic biopsy\n5. Rapid urease test",
    "A 40-year-old female patient presented with epigastric pain that started two weeks ago and intermittent dizziness that began a week ago. Her medical history includes taking painkillers (aceclofenac) for knee arthritis and anticoagulants for deep vein thrombosis. She also has a recent history of antibiotic use for bronchitis. Abdominal physical examination showed no abnormalities, but the rectal examination was positive. Vital signs were stable. Blood tests revealed a hemoglobin level of 8 g/dL. Gastroscopy did not show active bleeding, but an H1 ulcer was found in the antrum, and the rapid urease test was negative. Considering the patient needs to continue pain relief due to severe arthritis pain, what is the most effective treatment in this situation?\n1. Co-administration of an H2 receptor antagonist\n2. Switching to another COX-1 inhibitor and administering misoprostol\n3. Switching to a COX-2 inhibitor\n4. Switching to a COX-2 inhibitor and administering a PPI\n5. Switching to another COX-1 inhibitor",
    "What is the most common cause of gastric ulcers?\n1. Helicobacter pylori infection\n2. Stress\n3. Alcohol consumption\n4. Smoking",
    "Which of the following is NOT a major symptom of gastric ulcers?\n1. Heartburn\n2. Weight loss\n3. Cough\n4. Vomiting",
    "What is the most useful test for diagnosing gastric ulcers?\n1. Blood test\n2. Gastroscopy\n3. Ultrasound\n4. X-ray",
    "Which non-invasive test is appropriate for diagnosing Helicobacter pylori infection?\n1. Blood test\n2. Histology\n3. Urea breath test\n4. Gastroscopy",
    "Which of the following is NOT used in the treatment of gastric ulcers?\n1. PPI (Proton Pump Inhibitor)\n2. H2 receptor antagonist\n3. Antibiotics\n4. Insulin",
    "What is the recommended dietary habit for patients with gastric ulcers?\n1. Eating spicy foods\n2. Regular meals\n3. Fasting\n4. Encouraging alcohol and smoking",
    "Why can NSAIDs (Non-Steroidal Anti-Inflammatory Drugs) cause gastric ulcers?\n1. Because they promote digestion\n2. Because they inhibit the production of prostaglandins that protect the gastric mucosa\n3. Because they enhance immune function\n4. Because they increase blood pressure",
    "What is the first thing to do when acute bleeding occurs in a gastric ulcer patient?\n1. Provide intravenous fluids\n2. Administer antibiotics\n3. Surgery\n4. Administer painkillers",
    "What lifestyle habit is recommended to prevent the recurrence of gastric ulcers after treatment?\n1. Regular exercise and stress management\n2. Eating spicy foods\n3. Skipping meals\n4. Keep smoking",
    "Which medication class is most effective in reducing gastric acid secretion for ulcer treatment?\n1. Antacids\n2. Proton Pump Inhibitors (PPIs)\n3. Antibiotics\n4. Analgesics",
    "What is the most common site for gastric ulcers?\n1. Fundus of the stomach\n2. Antrum of the stomach\n3. Cardia of the stomach\n4. Duodenum",
    "Which of the following is a typical finding in a patient with a bleeding gastric ulcer?\n1. Increased hemoglobin levels\n2. Melena (black, tarry stools)\n3. Jaundice\n4. Polyuria",
    "Which treatment approach is NOT commonly used for managing gastric ulcers?\n1. Antacids for immediate relief\n2. Lifestyle modifications\n3. Corticosteroids\n4. Eradication of H. pylori infection",
    "50-year-old male presents with burning epigastric pain that worsens with meals and is relieved by antacids. He has a history of smoking and occasional alcohol use. Physical examination reveals mild epigastric tenderness. What is the most likely diagnosis, and what diagnostic test should be performed next?. Most likely diagnosis:\n1. Gastritis\n2. Gastric ulcer\n3. GERD\n4. Pancreatitis",
    "50-year-old male presents with burning epigastric pain that worsens with meals and is relieved by antacids. He has a history of smoking and occasional alcohol use. Physical examination reveals mild epigastric tenderness. What is the most likely diagnosis, and what diagnostic test should be performed next?. Most likely Diagnostic test:\n1. Abdominal ultrasound\n2. Blood test\n3. Upper endoscopy\n4. Barium swallow",
    "A 45-year-old female reports a 3-month history of intermittent, dull, gnawing pain in the upper abdomen. She also mentions nausea and occasional vomiting. She has been taking NSAIDs for chronic back pain. Her stool test is positive for occult blood. What is the most likely cause of her symptoms, and what should be the initial management step?\n Most likely cause:\n1. Gallstones\n2. NSAID-induced gastric ulcer\n3. Irritable bowel syndrome\n4. Pancreatitis",
    "A 45-year-old female reports a 3-month history of intermittent, dull, gnawing pain in the upper abdomen. She also mentions nausea and occasional vomiting. She has been taking NSAIDs for chronic back pain. Her stool test is positive for occult blood. What is the most likely cause of her symptoms, and what should be the initial management step?\nInitial management step:\n1. Continue NSAIDs and add antacids\n2. Discontinue NSAIDs and start a Proton Pump Inhibitor (PPI)\n3. Perform surgery\n4. Start antibiotics",
    "A 60-year-old male presents with sudden onset of severe abdominal pain, rigidity, and a history of chronic gastric ulcer. He appears pale and diaphoretic. Vital signs indicate hypotension and tachycardia. What complication should be suspected, and what is the immediate management?\n Suspected complication:\n1. Gastric cancer\n2. Perforated gastric ulcer\n3. Gallbladder disease\n4. Pancreatitis\n",
    "A 60-year-old male presents with sudden onset of severe abdominal pain, rigidity, and a history of chronic gastric ulcer. He appears pale and diaphoretic. Vital signs indicate hypotension and tachycardia. What complication should be suspected, and what is the immediate management?\n Immediate management:\n1. Administer painkillers\n2. Emergency surgery and resuscitation with fluids\n3. Perform blood tests\n4. Start antibiotics",
    "A 55-year-old male with a history of gastric ulcers treated in the past presents with weight loss, anorexia, and persistent epigastric pain unresponsive to PPIs. An upper endoscopy reveals an ulcer with irregular, raised borders in the gastric antrum. Biopsy results are pending. What is the most concerning potential diagnosis, and what should be the next step in management?\n Most concerning potential diagnosis:\n1. Benign gastric ulcer\n2. Gastric cancer\n3. GERD\n4. Gastritis",
    "A 55-year-old male with a history of gastric ulcers treated in the past presents with weight loss, anorexia, and persistent epigastric pain unresponsive to PPIs. An upper endoscopy reveals an ulcer with irregular, raised borders in the gastric antrum. Biopsy results are pending. What is the most concerning potential diagnosis, and what should be the next step in management?\n Next step in management:\n1. Prescribe antibiotics\n2. Increase PPI dosage\n3. Await biopsy results and consider referral to oncology\n4. Start corticosteroids",
    "A 48-year-old female presents with epigastric pain and black, tarry stools for the past two days. She has a history of rheumatoid arthritis and has been on long-term NSAID therapy. On examination, she appears pale and has a heart rate of 110 bpm. Her hemoglobin level is 7 g/dL. What is the most likely diagnosis, and what is the immediate management step?\n Most likely diagnosis:\n1. GERD\n2. Peptic ulcer with bleeding\n3. Gastric cancer\n4. Pancreatitis",
    "A 48-year-old female presents with epigastric pain and black, tarry stools for the past two days. She has a history of rheumatoid arthritis and has been on long-term NSAID therapy. On examination, she appears pale and has a heart rate of 110 bpm. Her hemoglobin level is 7 g/dL. What is the most likely diagnosis, and what is the immediate management step?\n Immediate management step:\n1. Administer painkillers\n2. Immediate blood transfusion and endoscopic intervention\n3. Start a high-fiber diet\n4. Increase NSAID dosage"
]


In [10]:
# Prepare storage for answers
answers = []

In [11]:
for question in questions:
    result = graph.invoke({"question": question})
    answers.append(result['answer'])

In [12]:
# Save to Excel and CSV
data = {"Question": questions, "Answer": answers}
df = pd.DataFrame(data)

output_excel_path = "output_llama_rag_options_excel.xlsx"
output_csv_path = "output_llama_rag_options_csv.csv"

df.to_excel(output_excel_path, index=False)
df.to_csv(output_csv_path, index=False)

print(f"Answers saved to {output_excel_path} and {output_csv_path}")

Answers saved to output_llama_rag_options_excel.xlsx and output_llama_rag_options_csv.csv


In [13]:
df

,Question,Answer
0,"The triple therapy consisting of PPI, clarithr...","Based on the context provided, the most approp..."
1,A 40-year-old female patient presented with ep...,"Based on the provided context, the most effect..."
2,What is the most common cause of gastric ulcer...,"Based on the provided context, I would conclud..."
3,Which of the following is NOT a major symptom ...,"Based on the provided context, I would say tha..."
4,What is the most useful test for diagnosing ga...,"Based on the provided context, the most useful..."
5,Which non-invasive test is appropriate for dia...,"Based on the context provided, the correct opt..."
6,Which of the following is NOT used in the trea...,"Based on the provided context, I would say tha..."
7,What is the recommended dietary habit for pati...,"Based on the provided context, I would recomme..."
8,Why can NSAIDs (Non-Steroidal Anti-Inflammator...,"Based on the provided context, the correct ans..."
9,What is the first thing to do when acute bleed...,"Based on the provided context, the correct ans..."
